In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Check if all entries are non-negative (positive or 0) elements
def all_nonnegative(df):
    return np.all(df.values >= 0)
# Check if all entries are 

In [ ]:
np.ll